In [1]:
import numpy as np
import pandas as pd
import cv2
import sys
from time import clock
import matplotlib.pyplot as plt
import os.path

In [2]:
from numpy import genfromtxt
my_data = genfromtxt('stefanie.csv', delimiter=',')
Y = []
for i in my_data:
    Y.append(1)

In [3]:
print(np.array(Y).shape)

(104,)


In [4]:
jimmy = genfromtxt('jonathan_jimmy.csv', delimiter=',')
for i in jimmy:
    Y.append(3)
X = np.vstack((my_data, jimmy))

In [ ]:
delaGala = genfromtxt('delaGala.csv', delimiter=',')
for i in delaGala:
    Y.append(2)
X = np.vstack((X, delaGala))

In [ ]:
harry = genfromtxt('harry_marquez.csv', delimiter=',')
for i in harry:
    Y.append(0)
X = np.vstack((X, harry))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
clf = RandomForestClassifier(n_estimators=500, max_depth=20, random_state=77)

In [ ]:
clf.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=77, verbose=0, warm_start=False)

In [ ]:
answer = clf.predict(X)

In [ ]:
precision = 0
for i in range(0, len(answer)):
    if answer[i] == Y[i]:
        precision+=1
precision / len(Y)

1.0

In [ ]:
from sklearn.metrics import f1_score
f1_score(Y, answer, average='weighted')

1.0

In [ ]:
f1_score(Y, answer, average='macro')

1.0

In [ ]:
f1_score(Y, answer, average='micro')

1.0

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X, Y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.96 (+/- 0.11)


In [ ]:
def getName(result):
    if result == 1:
        return 'Stefanie'
    elif result == 3:
        return 'Jimmy'
    elif result == 2:
        return 'deLaGala'
    elif result == 0:
        return 'Harry'
    else:
        return 'none'

In [ ]:
cascPath =  "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

cap = cv2.VideoCapture(0)

a = 0
count = 0
while True:
    # Capture frame-by-frame
    #ret, frame = video_capture.read()
    ret, frame = cap.read()
    b = clock()
    if  ret==False:
        print("ERROR: No device found")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        #flags=cv2.cv.CV_HAAR_SCALE_IMAGE
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    
    if b - a > 1:
        # Draw a rectangle around the faces
        for (x, y, w, h) in faces:
            cv2.rectangle(gray, (x, y), (x+w, y+h), (0, 255, 0), 2)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                imagen = cv2.resize(gray[y:y+h,x:x+w],(100,100), interpolation = cv2.INTER_CUBIC)
                #imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
                imagen = np.array(imagen)
                imagen = np.ravel(imagen)
                temp = []
                temp.append(np.asarray(imagen))
                temp = np.asarray(temp)
                name = getName(clf.predict(temp)[0])
                cv2.putText(img = gray, text = name, org = (int((x+w)/2),(y+h)), fontFace = cv2.FONT_HERSHEY_DUPLEX, fontScale = 3, color = (int(0), int(255), int(0)))
            count += 1

        # Display the resulting frame
        cv2.imshow('Video', gray)
        a = b

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()